<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# NAML: Neural News Recommendation with Attentive Multi-View Learning
NAML \[1\] is a multi-view news recommendation approach. The core of NAML is a news encoder and a user encoder. The newsencoder is composed of a title encoder, a body encoder, a vert encoder and a subvert encoder. The CNN-based title encoder and body encoder learn title and body representations by capturing words semantic information. After getting news title, body, vert and subvert representations, an attention network is used to aggregate those vectors. In the user encoder, we learn representations of users from their browsed news. Besides, we apply additive attention to learn more informative news and user representations by selecting important words and news.

## Properties of NAML:
- NAML is a multi-view neural news recommendation approach.
- It uses news title, news body, news vert and news subvert to get news repersentations. And it uses user historical behaviors to learn user representations.
- NAML uses additive attention to learn informative news and user representations by selecting important words and news.

## Data format:

### train data
One simple example: <br>

`1 0 0 0 0 Impression:0 User:502 CandidateTitle0:17917,36557,47926,32224,24113,48923,19086,5636,3703,0... CandidateBody0:17024,53305,8832,29800,9787,4068,48731,48923,19086,38699,5766,22487,38336,29800,8548,39128,33457,7789,
30543,7482,8548,49004,53305,22999,32747,21103,11799,5766,4868,17115,7482,15118,48731,2025,7789,23336,7789,48731,19086,
10630,11128,36557,3703,47354,611,7789,19086,5636,51521,30706... CandidateVert0:14... CandidateSubvert0:219... ClickedTitle0:48,33405,35198,5969,5636,35845,850,48731,46799,24113... ClickedBody0:36557,67,34519,24113,8548,48,33405,35198,5969,14340,7053,850,8823,9498,46799,24113,12506,32747,31130,
3074,48731,20869,14264,38289,37310,7789,36557,34967,48731,36916,23321,3595,48731,47354,4868,15719,7482,12771,50693,
47354,17523,48,20918,17900,35198,48731,20869,1220,14264,7789... ClickedVert0:14... ClickedSubvert0:99... `
<br>

In general, each line in data file represents one positive instance and n negative instances in a same impression. The format is like: <br>

`[label0] ... [labeln] [Impression:i] [User:u] [CandidateTitle0:w1,w2,w3,...] ... [CandidateBody0:w1,w2 ..] ... [CandidateVert0:v] ... [CandidateSubvert0:s] ... [ClickedTitle0:w1,w2,w3,...] ... [ClickedBody0:w1,w2,w3,...] ... [ClickedVert0:v] ... [ClickedSubvert0:s] ...`

<br>

It contains several parts seperated by space, i.e. label part, Impression part `<impresison id>`, User part `<user id>`, CandidateNews part, ClickedHistory part. CandidateNews part describes the target news article we are going to score in this instance. It is represented by (aligned) title words, body words, news vert index and subvert index. To take a quick example, a news title may be : `Trump to deliver State of the Union address next week` , then the title words value may be `CandidateTitlei:34,45,334,23,12,987,3456,111,456,432`. <br>
ClickedHistory describe the k-th news article the user ever clicked and the format is the same as candidate news. Every clicked news has title words, body words, vert and subvert. We use a fixed length to describe an article, if the title or body is less than the fixed length, just pad it with zeros.

### test data
One simple example: <br>
`1 Impression:0 User:1529 CandidateTitle0:5327,18658,13846,6439,611,50611,0,0,0,0 CandidateBody0:13846,3197,27902,225,5327,45008,29145,7789,509,7395,11502,36557,13846,23680,26492,38072,20507,5636,
4247,32747,50132,7482,41049,32747,43022,50611,35979,7789,1191,36557,52870,21622,48148,42737,48731,36557,13846,23680,
13173,7482,13848,38072,20507,7789,41675,36875,51461,12348,21045,42160 CandidateVert0:14 CandidateSubvert0:19 ClickedTitle0:9079,3703,32747,8546,19377,50184,32747,24026,40010,49754 ... ClickedBody0:26061,48731,8576,7789,8683,9079,5636,45084,46805,3703,509,43036,11502,28883,9498,18450,32747,8546,33405,
35647,50184,7482,41143,8220,43618,38072,35198,43390,28057,32552,45245,10764,16247,4221,41038,36557,43683,46805,7789,
29727,2179,51003,34797,897,21045,12974,23382,46287,48731,15206 ... ClickedVert0:14 ... ClickedSubvert0:219 ...`
<br>

In general, each line in data file represents one instance. The format is like: <br>

`[label] [Impression:i] [User:u] [CandidateTitle0:w1,w2,w3,...] [CandidateBody0:w1,w2,w3,...] [CandidateVert0:v] [CandidateSubvert0:s] [ClickedTitle0:w1,w2,w3,...] ... [ClickedBody0:w1,w2,w3,...] ... [ClickedVert0:v] ... [ClickedSubvert0:s] ...`
<br>

## Global settings and imports

In [1]:
import sys
sys.path.append("../../")
from reco_utils.recommender.deeprec.deeprec_utils import download_deeprec_resources  
from reco_utils.recommender.newsrec.newsrec_utils import *
from reco_utils.recommender.newsrec.models.naml import *
from reco_utils.recommender.newsrec.IO.naml_iterator import *
import papermill as pm

/data/anaconda/envs/reco_base/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/data/anaconda/envs/reco_base/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/data/anaconda/envs/reco_base/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/data/anaconda/envs/reco_base/lib/python3.6/site-packages/ten

## Download and load data

In [2]:
data_path = '../../tests/resources/newsrec/naml'
yaml_file = os.path.join(data_path, r'naml.yaml')
train_file = os.path.join(data_path, r'train.txt')
valid_file = os.path.join(data_path, r'test.txt')
wordEmb_file = os.path.join(data_path, r'embedding.npy')
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.blob.core.windows.net/newsrec/', data_path, 'naml.zip')

100%|██████████| 72.6k/72.6k [00:06<00:00, 11.7kKB/s]


## Create hyper-parameters

In [3]:
epoch=5

In [4]:
hparams = prepare_hparams(yaml_file, wordEmb_file=wordEmb_file, epochs=epoch)
print(hparams)

[('attention_hidden_dim', 100), ('batch_size', 64), ('body_size', 50), ('cnn_activation', 'relu'), ('data_format', 'naml'), ('dense_activation', 'relu'), ('doc_size', None), ('dropout', 0.2), ('epochs', 5), ('filter_num', 200), ('gru_unit', 400), ('head_dim', 100), ('head_num', 4), ('his_size', 50), ('iterator_type', None), ('learning_rate', 0.0001), ('loss', 'cross_entropy_loss'), ('metrics', ['group_auc', 'mean_mrr', 'ndcg@5;10']), ('npratio', 4), ('optimizer', 'adam'), ('show_step', 100000), ('subvert_emb_dim', 100), ('subvert_num', 249), ('title_size', 10), ('type', 'ini'), ('user_emb_dim', 50), ('user_num', 10329), ('vert_emb_dim', 100), ('vert_num', 17), ('window_size', 3), ('wordEmb_file', '../../tests/resources/newsrec/naml/embedding.npy'), ('word_emb_dim', 100), ('word_size', 54071)]


In [5]:
train_iterator = NAMLTrainIterator
test_iterator = NAMLTestIterator

## Train the NAML model

In [6]:
model = NAMLModel(hparams, train_iterator, test_iterator)

In [7]:
print(model.run_eval(valid_file))

{'group_auc': 0.5105, 'mean_mrr': 0.168, 'group_ndcg@5': 0.1637, 'group_ndcg@10': 0.2232}


In [8]:
model.fit(train_file, valid_file)

/data/anaconda/envs/reco_base/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


at epoch 1
train info: logloss loss:1.5874547812403466
eval info: group_auc:0.5487, group_ndcg@10:0.2343, group_ndcg@5:0.1789, mean_mrr:0.1747
at epoch 1 , train time: 45.1 eval time: 36.3
at epoch 2
train info: logloss loss:1.5494758206970838
eval info: group_auc:0.5592, group_ndcg@10:0.2501, group_ndcg@5:0.1815, mean_mrr:0.1854
at epoch 2 , train time: 39.3 eval time: 36.0
at epoch 3
train info: logloss loss:1.5116341741717592
eval info: group_auc:0.5599, group_ndcg@10:0.2526, group_ndcg@5:0.1873, mean_mrr:0.1851
at epoch 3 , train time: 39.5 eval time: 35.3
at epoch 4
train info: logloss loss:1.4747365990463568
eval info: group_auc:0.5561, group_ndcg@10:0.2551, group_ndcg@5:0.1847, mean_mrr:0.186
at epoch 4 , train time: 39.3 eval time: 36.2
at epoch 5
train info: logloss loss:1.440334933631274
eval info: group_auc:0.553, group_ndcg@10:0.2539, group_ndcg@5:0.1867, mean_mrr:0.1878
at epoch 5 , train time: 39.3 eval time: 35.9


In [9]:
print(model.run_eval(valid_file))

{'group_auc': 0.553, 'mean_mrr': 0.1878, 'group_ndcg@5': 0.1867, 'group_ndcg@10': 0.2539}


## Reference
\[1\] Chuhan Wu, Fangzhao Wu, Mingxiao An, Jianqiang Huang, Yongfeng Huang and Xing Xie: Neural News Recommendation with Attentive Multi-View Learning, IJCAI 2019<br>